In [4]:
import pandas as pd

df  = pd.read_csv("mempool.csv")

def transaction_sort(df, fee_max, min_wt):
    df = df.sort_values([fee_max, min_wt], ascending=[False, True]).reset_index(drop=True)
    return df

def list_checker(x):
    if str(x) in final_set_of_txids:
        return True
    else:
        return False

def parent_checker(x):
    if str(x[3]) != "nan":
        parent_list = str(x[3]).split(";")
        for i in parent_list:
            if(list_checker(i)):
                continue
            else:
                txnindex = df[df['tx_id'] == i].index.item()
                k = df.loc[txnindex]
                check_add_txn(k)

def weight_checker(x):
    if min_weight + x['weight'] <= highest_weight:
        return True
    else:
        return False

def add_to_block(x):
    global min_weight
    txnID = x[0]
    weight = x[2]
    min_weight += weight
    final_set_of_txids.append(txnID)

def check_add_txn(x):
    if(weight_checker(x)):
        if(not list_checker(x)):
            parent_checker(x)
            if(weight_checker(x)):
                add_to_block(x)


def Main(df):
    sorted_transactions = transaction_sort(df, "fee", "weight")
    for i in range(len(sorted_transactions)):
        txnVar =  sorted_transactions.loc[i]
        check_add_txn(txnVar)


def write_to_file(fin_list):
    file = open("block.txt","a")
    for i in fin_list:
        file.write(str(i) + '\n')
    file.close()


highest_weight = 4000000
min_weight = 0
final_set_of_txids = []

data = df
Main(data)

write_to_file(final_set_of_txids)